## Data Preprocessing Pipeline

Data is hierarchically organized as follows: 'root/make_id/model_id/released_year/image_name.jpg'. Root is the 'image' folder of the CompCars dataset.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Subset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm, trange

# from custom files
from dataset import CompCarsImageFolder
from models import BasicBlock, BottleneckBlock, ResNet, resnet_cfg
from models import train, validate
from utils import *

## Configuration

In [ ]:
### Set root to the image folder of CompCars dataset
# root = 'data/image'  # TODO: ADAPT TO YOUR FOLDER STRUCTURE
root = '../cars_data/data/image'


### Hyperparam configuration
params = {                  ## Training Params (taken from original resnet paper: https://arxiv.org/pdf/1512.03385)
    'epoch_num': 10,        # number of epochs
    'lr': 1e-1,             # Learning Rate
    'weight_decay': 1e-4,   # L2 Penalty
    'batch_size': 256,      # batch size
    'momentum': 0.9,
    
    'hierarchy': 0,         # Choose 0 for manufacturer classification, 1 for model classifciation
    'val_split': 0.2,        # Fraction of validation holdout
    
    'resnet': resnet_cfg['resnet18']  # Resnet model used
}

### Device
if torch.cuda.is_available():
    params["device"] = torch.device("cuda")   # option for NVIDIA GPUs
elif torch.backends.mps.is_available():
    params["device"] = torch.device("mps")    # option for Mac M-series chips (GPUs)
else:
    params["device"] = torch.device("cpu")    # default option if none of the above devices are available

print("Device: {}".format(params["device"]))

### Transforms
# TODO: Adapt transforms to our data set
# TODO: maybe use v2 transforms: https://pytorch.org/vision/stable/transforms.html
data_transforms = {
        'train': transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # TODO: find normalization for CompCars dataset
        ]),
        'val': transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # TODO: find normalization for CompCars dataset
        ])
}

In [ ]:
class WrapperDataset:
    def __init__(self, dataset, transform=None, target_transform=None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        image, label = self.dataset[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.dataset)

def train_val_dataset(dataset, val_split=params['val_split']):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

### Save total CompCars dataset in a DataFolder class

In [ ]:
# hierarchy=0 -> manufacturer classification; hierarchy=1 -> model classification
total_set = CompCarsImageFolder(root, hierarchy=params['hierarchy'])  # Adjust hierarchy as needed
print(total_set.classes)
print(len(total_set.classes))

### Split in training and validation data

In [ ]:
datasets = train_val_dataset(total_set)

wrapped_datasets = {
    'train': WrapperDataset(datasets['train'], transform=data_transforms['train']),
    'val': WrapperDataset(datasets['val'], transform=data_transforms['val'])
}

dataloaders = {
    'train': DataLoader(wrapped_datasets['train'], batch_size=params['batch_size'], shuffle=True, num_workers=4),
    'val': DataLoader(wrapped_datasets['val'], batch_size=params['batch_size'], shuffle=False, num_workers=4)
}

print(f"Total dataset size: {len(total_set)}")
print(f"Training dataset size: {len(datasets['train'])}")
print(f"Validation dataset size: {len(datasets['val'])}")

x, y = next(iter(dataloaders['train']))
print(f"Batch of images shape: {x.shape}")
print(f"Batch of labels shape: {y.shape}")

## Visualize Data-Set

In [ ]:
# TODO

## Set-Up Training

In [ ]:
# Set up resnet model
resnet = ResNet(params['resnet']['block'], params['resnet']['layers'], 
                len(total_set.classes)).to(params['device'])

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD( #SGD used in original resnet paper
    resnet.parameters(), 
    lr=params['lr'], 
    weight_decay=params['weight_decay'], 
    momentum=params['momentum']
)

## Training Loop

In [ ]:
# Training and Validation Loops
train_losses, validation_losses, train_acc, validation_acc = list(), list(), list(), list()

# Just some fancy progress bars
pbar_epoch = trange(params["epoch_num"], desc="Training")
pbar_inside_epoch = tqdm(total = (len(dataloaders['train'])+len(dataloaders['val'])), desc="Training and validation per epoch", position=1, leave=True)

# Stop the training phase in case there is no improvement
early_stopper = EarlyStopper(patience=10, min_delta=0.1)

for epoch in pbar_epoch:
    pbar_inside_epoch.reset()
    
    train_results = train(dataloaders['train'], resnet, epoch, criterion, optimizer, params["device"], pbar=pbar_inside_epoch)
    train_losses.append(train_results[0])
    train_acc.append(1 - train_results[1])
    
    validation_results = validate(dataloaders['val'], resnet, epoch, criterion, params["device"], pbar=pbar_inside_epoch)
    validation_losses.append(validation_results[0])
    validation_acc.append(1 - validation_results[1])
    
    # Comment on the following lines if you don't want to stop early in case of no improvement
    if early_stopper.early_stop(validation_results[0]):
        params['epoch_num'] = epoch
        print("\n\nEarly stopping...")
        break

pbar_inside_epoch.close()

## Plot losses

In [ ]:
# Plotting the performance of the model in the training and validation phase

plots = [
    (np.arange(0, params["epoch_num"], 1), train_losses, "Train Loss"),
    (np.arange(0, params["epoch_num"], 1), validation_losses, "Validation Loss")
]

show_plot(plots, "Model Loss for Epoch", "Epoch", "Loss")

plots = [
    (np.arange(0, params["epoch_num"], 1), train_acc, "Train Error"),
    (np.arange(0, params["epoch_num"], 1), validation_acc, "Validation Error")
]

show_plot(plots, "Model Error for Epoch", "Epoch", "Error")